In [ ]:
import numpy as np
import torch
from NeuralNet import NeuralNetwork as Network

In [ ]:
Filters = 64
Layers = 10
HistoryDepth = 8
KernalSize = 3
targetDevice = "cpu"

model = Network(Filters, HistoryDepth + 1, Layers, kernal_size=KernalSize)

model.load_state_dict(torch.load('../../Models/HumanModels/64f10l/4.pt'))
model.to(targetDevice)
model.eval()

In [ ]:
example = np.zeros((1, HistoryDepth + 1, 15, 15), dtype=np.float32)
example = torch.tensor(example)

traced_script_module = torch.jit.trace(model, example)
traced_script_module.save("../../Models/HumanModels/traced_model.pt")